In [0]:
%pip install -U -qqq langchain_core langchain_databricks langchain_community
%restart_python

In [0]:
import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_databricks import ChatDatabricks
from databricks.sdk import WorkspaceClient
import os

In [0]:

# configure workspace tokens
w = WorkspaceClient()
os.environ["DATABRICKS_HOST"] = w.config.host
os.environ["DATABRICKS_TOKEN"] = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value

In [0]:
MODEL = "databricks-llama-4-maverick"
# "databricks-meta-llama-3-1-405b-instruct"

In [0]:
llm = ChatDatabricks(endpoint=MODEL)

def format_context(df: pd.DataFrame) -> str:
    """
    Converts the DataFrame into a JSON string to ensure all data is passed
    to the model without truncation. JSON is also a great format for structured data
    like you have in 'description_by_sections'.
    """
    return df.to_json(orient='records', indent=2)

def find_accessible_psychotherapists(location: str) -> pd.DataFrame:
  """
  Queries the Bright Initiative google maps dataset for psychotherapists on a specific location.
  """
  query = f"""
    SELECT * FROM `dais-hackathon-2025`.bright_initiative.google_maps_businesses where category = 'Psychotherapist' AND country = 'US' AND address LIKE '%{location}%'
  """
  return format_context(spark.sql(query).toPandas())

# Define the prompt template for the LLM
prompt_template = PromptTemplate.from_template(
  """
  You are a helpful assistant with the purpose of finding helpful mental health resources. Your goal is to summarize potential psychoterapist venues a user can get in touch.

  Summarize the information the user can use to get in touch with.

  If the person types the name of a city or location, you should only show the closest locations to the city or place the user sent

  Here is the JSON data:
  {context}
  """
)

llm = ChatDatabricks(endpoint=MODEL)

# This is our simple "agentic" chain
chain = (
    find_accessible_psychotherapists
    | prompt_template
    | llm
    | StrOutputParser()
)

# Let's run the chain for Chicago!
result = chain.invoke("Chicago")

print(result)